In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load the integrated and cleaned dataset
final_data = pd.read_csv("/Users/asr/Desktop/College/CapstoneProject/CapStoneProject_Group3/dataset/final_data.csv")

print("Final Data loaded successfully.")
print("Shape of final_data:", final_data.shape)
display(final_data.head())

Final Data loaded successfully.
Shape of final_data: (23918082, 37)


,listing_id,scrape_id,name,host_id,host_name,host_since,host_location,latitude,longitude,neighbourhood_cleansed,...,reviews_per_month,number_of_reviews,first_review,last_review,available,price_x,availability_30,availability_60,availability_90,availability_365
0,241032,20160104002432,Stylish Queen Anne Apartment,956883,Maija,2011-08-11,"Seattle, Washington, United States",47.636289,-122.371025,West Queen Anne,...,4.07,207,2011-11-01,2016-01-02,1,85.0,14,41,71,346
1,241032,20160104002432,Stylish Queen Anne Apartment,956883,Maija,2011-08-11,"Seattle, Washington, United States",47.636289,-122.371025,West Queen Anne,...,4.07,207,2011-11-01,2016-01-02,1,85.0,14,41,71,346
2,241032,20160104002432,Stylish Queen Anne Apartment,956883,Maija,2011-08-11,"Seattle, Washington, United States",47.636289,-122.371025,West Queen Anne,...,4.07,207,2011-11-01,2016-01-02,0,85.0,14,41,71,346
3,241032,20160104002432,Stylish Queen Anne Apartment,956883,Maija,2011-08-11,"Seattle, Washington, United States",47.636289,-122.371025,West Queen Anne,...,4.07,207,2011-11-01,2016-01-02,0,85.0,14,41,71,346
4,241032,20160104002432,Stylish Queen Anne Apartment,956883,Maija,2011-08-11,"Seattle, Washington, United States",47.636289,-122.371025,West Queen Anne,...,4.07,207,2011-11-01,2016-01-02,0,85.0,14,41,71,346


Local Python-Based Ingestion Pipeline

In [7]:
import os
import pandas as pd
from google.cloud import storage
from google.cloud import bigquery
import logging

In [11]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("data_ingestion_pipeline.log"),  # Save logs to a file
        logging.StreamHandler()  # Print logs to the console
    ]
)

In [27]:
# GCP credentials and configurations
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/asr/Desktop/College/CapstoneProject/CapStoneProject_Group3/capstoneproject-444019-7bad7df402b1.json"
PROJECT_ID = "capstoneproject-444019"
DATASET_NAME = "airbnb_dataset"
BUCKET_NAME = "airbnb-data-ingestion"

In [39]:
from google.cloud import storage

# Initialize a storage client
storage_client = storage.Client()

# Access the bucket
bucket = storage_client.bucket(BUCKET_NAME)

print(f"Bucket {BUCKET_NAME} is accessible!")

Bucket airbnb-data-ingestion is accessible!


In [ ]:
def upload_to_gcs(file_path, bucket_name, destination_blob_name):
    """Upload file to GCP Cloud Storage."""
    logging.info(f"Uploading {file_path} to GCS bucket {bucket_name}")
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(file_path)
    logging.info(f"File {file_path} uploaded to {destination_blob_name}.")

def load_data_to_bigquery(table_name, gcs_uri):
    """Load data from GCS to BigQuery."""
    logging.info(f"Loading data from GCS URI {gcs_uri} to BigQuery table {table_name}.")
    client = bigquery.Client()
    print("Connection successful")
    table_id = f"{PROJECT_ID}.{DATASET_NAME}.{table_name}"

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        # write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
        schema = [
            bigquery.SchemaField("listing_id", "INTEGER"),
            bigquery.SchemaField("scrape_id", "INTEGER"),
            bigquery.SchemaField("name", "STRING"),
            bigquery.SchemaField("host_id", "INTEGER"),
            bigquery.SchemaField("host_name", "STRING"),
            bigquery.SchemaField("host_since", "DATE"),
            bigquery.SchemaField("host_location", "STRING"),
            bigquery.SchemaField("latitude", "FLOAT"),
            bigquery.SchemaField("longitude", "FLOAT"),
            bigquery.SchemaField("neighbourhood_cleansed", "STRING"),
            bigquery.SchemaField("city", "STRING"),
            bigquery.SchemaField("state", "STRING"),
            bigquery.SchemaField("country", "STRING"),
            bigquery.SchemaField("property_type", "STRING"),
            bigquery.SchemaField("room_type", "STRING"),
            bigquery.SchemaField("accommodates", "INTEGER"),
            bigquery.SchemaField("bathrooms", "FLOAT"),
            bigquery.SchemaField("bedrooms", "FLOAT"),
            bigquery.SchemaField("beds", "FLOAT"),
            bigquery.SchemaField("price_y", "FLOAT"),
            bigquery.SchemaField("review_scores_rating", "FLOAT"),
            bigquery.SchemaField("review_scores_accuracy", "FLOAT"),
            bigquery.SchemaField("review_scores_cleanliness", "FLOAT"),
            bigquery.SchemaField("review_scores_checkin", "FLOAT"),
            bigquery.SchemaField("review_scores_communication", "FLOAT"),
            bigquery.SchemaField("review_scores_location", "FLOAT"),
            bigquery.SchemaField("review_scores_value", "FLOAT"),
            bigquery.SchemaField("reviews_per_month", "FLOAT"),
            bigquery.SchemaField("number_of_reviews", "INTEGER"),
            bigquery.SchemaField("first_review", "DATE"),
            bigquery.SchemaField("last_review", "DATE"),
            bigquery.SchemaField("available", "BOOLEAN"),
            bigquery.SchemaField("price_x", "FLOAT"),
            bigquery.SchemaField("availability_30", "INTEGER"),
            bigquery.SchemaField("availability_60", "INTEGER"),
            bigquery.SchemaField("availability_90", "INTEGER"),
            bigquery.SchemaField("availability_365", "INTEGER"),
        ],
    )
    print("Job config created!")
    print(gcs_uri, table_id, job_config)
    load_job = client.load_table_from_uri(
        gcs_uri, table_id, job_config=job_config
    )  
    load_job.result()  # Wait for the job to complete.

    logging.info(f"Table {table_name} loaded successfully.")

def process_and_ingest(file_path, table_name):
    """Process CSV, upload to GCS, and load into BigQuery."""
    try:
        # Upload CSV to GCS
        destination_blob_name = f"airbnb/{table_name}.csv"
        upload_to_gcs(file_path, BUCKET_NAME, destination_blob_name)

        # Load data from GCS to BigQuery
        gcs_uri = f"gs://{BUCKET_NAME}/{destination_blob_name}"
        load_data_to_bigquery(table_name, gcs_uri)

    except Exception as e:
        logging.error(f"Error processing {file_path}: {e}")


In [ ]:
files_and_tables = {
    "/Users/asr/Desktop/College/CapstoneProject/CapStoneProject_Group3/dataset/final_data.csv": "final_data"
}

for file_path, table_name in files_and_tables.items():
    process_and_ingest(file_path, table_name)

In [31]:
from google.cloud import bigquery
import os

# Set the service account key file explicitly
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/asr/Desktop/College/CapstoneProject/CapStoneProject_Group3/capstoneproject-444019-7bad7df402b1.json"

try:
    # Test BigQuery client
    client = bigquery.Client()
    print("Authentication successful. Connected to BigQuery.")
except Exception as e:
    print(f"Authentication failed: {e}")


Authentication successful. Connected to BigQuery.


In [48]:
import os
from google.cloud import bigquery

# Set the environment variable explicitly
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/asr/Desktop/College/CapstoneProject/CapStoneProject_Group3/capstoneproject-444019-7bad7df402b1.json"

# Test BigQuery client
try:
    client = bigquery.Client()
    datasets = list(client.list_datasets())  # List datasets in the project
    print("Authentication successful. Datasets in project:")
    for dataset in datasets:
        print(f" - {dataset.dataset_id}")
except Exception as e:
    print(f"Authentication failed: {e}")


Authentication successful. Datasets in project:
 - airbnb_dataset


In [ ]:
destination_blob_name = f"airbnb/{table_name}.csv"
gcs_uri = f"gs://{BUCKET_NAME}/{destination_blob_name}"
load_data_to_bigquery(table_name, gcs_uri)

In [ ]:
table_id = "capstoneproject-444019.airbnb_dataset.final_data"

In [58]:
client = bigquery.Client(project=PROJECT_ID)

# Define the table schema
schema = [
    bigquery.SchemaField("listing_id", "INTEGER"),
    bigquery.SchemaField("scrape_id", "INTEGER"),
    bigquery.SchemaField("name", "STRING"),
    bigquery.SchemaField("host_id", "INTEGER"),
    bigquery.SchemaField("host_name", "STRING"),
    bigquery.SchemaField("host_since", "DATE"),
    bigquery.SchemaField("host_location", "STRING"),
    bigquery.SchemaField("latitude", "FLOAT"),
    bigquery.SchemaField("longitude", "FLOAT"),
    bigquery.SchemaField("neighbourhood_cleansed", "STRING"),
    bigquery.SchemaField("city", "STRING"),
    bigquery.SchemaField("state", "STRING"),
    bigquery.SchemaField("country", "STRING"),
    bigquery.SchemaField("property_type", "STRING"),
    bigquery.SchemaField("room_type", "STRING"),
    bigquery.SchemaField("accommodates", "INTEGER"),
    bigquery.SchemaField("bathrooms", "FLOAT"),
    bigquery.SchemaField("bedrooms", "FLOAT"),
    bigquery.SchemaField("beds", "FLOAT"),
    bigquery.SchemaField("price_y", "FLOAT"),
    bigquery.SchemaField("review_scores_rating", "FLOAT"),
    bigquery.SchemaField("review_scores_accuracy", "FLOAT"),
    bigquery.SchemaField("review_scores_cleanliness", "FLOAT"),
    bigquery.SchemaField("review_scores_checkin", "FLOAT"),
    bigquery.SchemaField("review_scores_communication", "FLOAT"),
    bigquery.SchemaField("review_scores_location", "FLOAT"),
    bigquery.SchemaField("review_scores_value", "FLOAT"),
    bigquery.SchemaField("reviews_per_month", "FLOAT"),
    bigquery.SchemaField("number_of_reviews", "INTEGER"),
    bigquery.SchemaField("first_review", "DATE"),
    bigquery.SchemaField("last_review", "DATE"),
    bigquery.SchemaField("available", "BOOLEAN"),
    bigquery.SchemaField("price_x", "FLOAT"),
    bigquery.SchemaField("availability_30", "INTEGER"),
    bigquery.SchemaField("availability_60", "INTEGER"),
    bigquery.SchemaField("availability_90", "INTEGER"),
    bigquery.SchemaField("availability_365", "INTEGER"),
]

# Configure the load job with partitioning and clustering
job_config = bigquery.LoadJobConfig(
    schema=schema,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Skip header row
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  # Overwrite the table
    time_partitioning=bigquery.TimePartitioning(
        type_=bigquery.TimePartitioningType.DAY,
        field="last_review",  # Partition by 'last_review'
    ),
    clustering_fields=["neighbourhood_cleansed", "room_type", "property_type"],  # Clustering columns
)


In [ ]:
# Start the load job
print(f"Loading data into {table_id} from {gcs_uri}...")
load_job = client.load_table_from_uri(
    gcs_uri, table_id, job_config=job_config
)

# Wait for the job to complete
try:
    load_job.result()
    print(f"Data successfully loaded into {table_id}.")
except Exception as e:
    print(f"Error during load job: {e}")

# Confirm table details
table = client.get_table(table_id)
print(f"Loaded {table.num_rows} rows into {table.table_id}.")
print(f"Table is partitioned by: {table.time_partitioning.field}")
print(f"Table is clustered on: {table.clustering_fields}")